In [1]:
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset
import torch.nn.functional as F
import json

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/home/morris/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
DATA_PATH = "../data/fouriers/"

In [3]:
# frequency range to fit model on
MIN_FREQ, MAX_FREQ = 2000, 4000
SAMPLE_RATE = 48000
NYQUIST = SAMPLE_RATE / 2 + 1

class AudioDataset(Dataset):
    def __init__(self, min_freq, max_freq):
        assert min_freq < max_freq, "Common man.."

        self.files = os.listdir(DATA_PATH)

        sample_fourier = json.load(open(DATA_PATH + self.files[0], "r")) 
        n_freqs = len(sample_fourier["powers"])
        # indices that correspond to MIN_FREQ and MAX_FREQ
        self.min_i = int(min_freq * n_freqs / NYQUIST)
        self.max_i = int(max_freq * n_freqs / NYQUIST)
        # number of frequencies per sample
        self.n_freqs = self.max_i - self.min_i + 1

        self.data = []        
        self.load_data()

    def load_data(self):  
        
         
        for file in self.files:
            fourier = json.load(open(DATA_PATH + file, "r")) 
            self.data.append((torch.tensor(fourier["powers"][self.min_i:self.max_i+1]), fourier["label"]))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

dataset = AudioDataset(MIN_FREQ, MAX_FREQ)

In [4]:
class TwoLayerIntegrated(nn.Module):
    def __init__(self, input_dim):
        super(TwoLayerIntegrated, self).__init__()
        
        self.layer1 = nn.Linear(input_dim, 128)
        self.layer2 = nn.Linear(128, 1)
    
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.sigmoid(self.layer2(x))
        
        return x

In [5]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm

# Set random seed for reproducibility
torch.manual_seed(42)

# Parameters
batch_size = 256
num_epochs = 200
validation_split = 0.2

# Create dataset and dataloader
dataset_size = len(dataset)
val_size = int(validation_split * dataset_size)
train_size = dataset_size - val_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

model = TwoLayerIntegrated(dataset.n_freqs).to(device)

# Define loss and optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters())

# Lists to keep track of losses and accuracy for plotting
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

def calculate_accuracy(predictions, targets):
    rounded_preds = torch.round(predictions)
    correct = (rounded_preds == targets).sum().item()
    total = targets.size(0)
    return correct / total

for epoch in tqdm(range(num_epochs)):
    # Training phase
    model.train()
    train_loss_sum = 0.0
    correct_train = 0
    total_train = 0
    for batch_waveforms, batch_labels in train_dataloader:
        batch_waveforms = batch_waveforms.to(device)
        batch_labels = batch_labels.to(device)
        optimizer.zero_grad()
        batch_outputs = model(batch_waveforms)
        loss = criterion(batch_outputs, batch_labels.unsqueeze(1).float())
        loss.backward()
        optimizer.step()
        train_loss_sum += loss.item()

        batch_predictions = torch.round(batch_outputs)
        correct_train += (batch_predictions == batch_labels.unsqueeze(1)).sum().item()
        total_train += batch_labels.size(0)
    
    train_loss_avg = train_loss_sum / len(train_dataloader)
    train_acc = correct_train / total_train
    train_losses.append(train_loss_avg)
    train_accuracies.append(train_acc)

    # Validation phase
    model.eval()
    val_loss_sum = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for batch_waveforms, batch_labels in val_dataloader:
            batch_waveforms = batch_waveforms.to(device)
            batch_labels = batch_labels.to(device)
            batch_outputs = model(batch_waveforms)
            loss = criterion(batch_outputs, batch_labels.unsqueeze(1).float())
            val_loss_sum += loss.item()

            batch_predictions = torch.round(batch_outputs)
            correct_val += (batch_predictions == batch_labels.unsqueeze(1)).sum().item()
            total_val += batch_labels.size(0)
    
    val_loss_avg = val_loss_sum / len(val_dataloader)
    val_acc = correct_val / total_val
    val_losses.append(val_loss_avg)
    val_accuracies.append(val_acc)

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}] - "
            f"Train Loss: {train_loss_avg:.4f}, Train Acc: {train_acc:.4f}, "
            f"Val Loss: {val_loss_avg:.4f}, Val Acc: {val_acc:.4f}")

# Save the trained model
torch.save(model.state_dict(),  "../models/whistle_detection.pth")

sample_input = torch.randn(1, dataset.n_freqs)
torch.onnx.export(model, sample_input.to(device), "../models/whistle_detection.onnx") 

  6%|▋         | 13/200 [00:00<00:12, 15.09it/s]

Epoch [10/200] - Train Loss: 0.1685, Train Acc: 0.9667, Val Loss: 0.1777, Val Acc: 0.9740


 10%|█         | 21/200 [00:01<00:13, 13.62it/s]

Epoch [20/200] - Train Loss: 0.0993, Train Acc: 0.9724, Val Loss: 0.1228, Val Acc: 0.9761


 16%|█▌        | 31/200 [00:02<00:11, 15.09it/s]

...

Epoch [180/200] - Train Loss: 0.0425, Train Acc: 0.9871, Val Loss: 0.0440, Val Acc: 0.9844


 96%|█████████▋| 193/200 [00:13<00:00, 15.47it/s]

Epoch [190/200] - Train Loss: 0.0566, Train Acc: 0.9875, Val Loss: 0.1243, Val Acc: 0.9816


100%|██████████| 200/200 [00:13<00:00, 14.68it/s]


Epoch [200/200] - Train Loss: 0.0567, Train Acc: 0.9893, Val Loss: 0.0947, Val Acc: 0.9830
